In [250]:
import tensorflow_datasets as tfds
pcam, pcam_info = tfds.load("patch_camelyon", with_info=True)
print(pcam_info)

tfds.core.DatasetInfo(
    name='patch_camelyon',
    full_name='patch_camelyon/2.0.0',
    description="""
    The PatchCamelyon benchmark is a new and challenging image classification
    dataset. It consists of 327.680 color images (96 x 96px) extracted from
    histopathologic scans of lymph node sections. Each image is annoted with a
    binary label indicating presence of metastatic tissue. PCam provides a new
    benchmark for machine learning models: bigger than CIFAR10, smaller than
    Imagenet, trainable on a single GPU.
    """,
    homepage='https://patchcamelyon.grand-challenge.org/',
    data_path='/Users/melody.zhao/tensorflow_datasets/patch_camelyon/2.0.0',
    download_size=7.48 GiB,
    dataset_size=7.06 GiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(96, 96, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disabl

In [257]:
from PIL import Image
import PIL.Image


In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Import TensorFlow and relevant Keras classes to setup the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
input_img = Input(shape=(96,96,3))

# Now we define the layers of the convolutional network: three blocks of two convolutional layers and a max-pool layer.
x = Conv2D(16, (3, 3), padding='valid', activation='relu')(input_img)
x = Conv2D(16, (3, 3), padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
x = Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
x = Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

# Now we flatten the output from a 4D to a 2D tensor to be able to use fully-connected (dense) layers for the final
# classification part. Here we also use a bit of dropout for regularization. The last layer uses a softmax to obtain class
# likelihoods (i.e. metastasis vs. non-metastasis)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
predictions = Dense(2, activation='softmax')(x)

# Now we define the inputs/outputs of the model and setup the optimizer. In this case we use regular stochastic gradient
# descent with Nesterov momentum. The loss we use is cross-entropy and we would like to output accuracy as an additional metric.
model = Model(inputs=input_img, outputs=predictions)
sgd_opt = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
model.compile(optimizer=sgd_opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 94, 94, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 42, 42, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 32)        0     

/Users/melody.zhao/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [5]:
def preprocess_content(img):
    max_dim = 512
#     img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [72]:
pcam['train']

<PrefetchDataset shapes: {id: (), image: (96, 96, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>

In [252]:
tf.executing_eagerly()


True

In [179]:
tf.config.run_functions_eagerly(True)


In [253]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [254]:
# To keep the dataset size small PatchCAMELYON is stored as int8 patches. 
# For network training we need float32 and we want to normalize between 0 and 1. 
# The function below performs this task.
import tensorflow_hub as hub
import glob
import functools
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
style_images = glob.glob("/Users/melody.zhao/Desktop/CS153-Computer-Vision/style_transfer_imgs/*")
style_image = load_img(style_images[2])

def convert_sample(sample):
    img, label = sample['image'], sample['label'] 
    max_dim = 512
#     img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    print(shape)
#     long_dim = max(shape)
    long_dim = functools.reduce(shape, lambda x, y: x if x > y else y)
#     scale = max_dim / long_dim

#     new_shape = tf.cast(shape * scale, tf.int32)

#     img = tf.image.resize(img, new_shape)
#     img = img[tf.newaxis, :]
#     img = hub_model(tf.constant(img), tf.constant(style_image))[0]
#     img = tensor_to_image(stylized_image_)
#     img = tf.image.convert_image_dtype(img, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return img, label

In [334]:
def preprocess_content(img):
    max_dim = 512
#     img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    
    ######### newly added
    image = hub_model(tf.constant(img), tf.constant(style_image))[0]
    image = tensor_to_image(image)
    print(image)
    image = image.resize((96,96))
    image = tf.keras.preprocessing.image.img_to_array(image)
    print('shape', image.shape)
    
    image = tf.image.convert_image_dtype(image, tf.float32)
    print('shape2', image.shape)
    return image

In [323]:
def convert_sample(sample):
    image, label = sample['image'], sample['label'] 
#     image = preprocess_content(image)
#     image = hub_model(tf.constant(image), tf.constant(style_image))[0]
#     image = tensor_to_image(stylized_image_)
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

In [299]:
list_train = list(pcam['train'])[:100]

In [316]:
img = preprocess_content(list_train[0]['image'])

In [301]:
list_img = [list_train[i]['image'] for i in range(len(list_train))]

In [302]:
list_label = [list_train[i]['label'] for i in range(len(list_train))]

In [335]:
list_img_convert = []
for i, img in enumerate(list_img):
#     print(preprocess_content(img))
    list_img_convert.append(preprocess_content(img))
    

<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF6BB081F0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD5A77CDF0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF54FB2820>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD5A77CDF0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF6BB08B50>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD5A77CDF0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF6BB08B50>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD5A77CDF0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF840CE6D0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF91421EE0>
shape (96, 96, 3)
shape2 (9

<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD58ADBC70>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD58ADBC70>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD58ADBC70>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD58ADBC70>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF94290250>
shape (96, 96, 3)
shape2 (9

In [336]:
list_img_convert[0]

<tf.Tensor: shape=(96, 96, 3), dtype=float32, numpy=
array([[[104., 137., 190.],
        [100., 147., 193.],
        [111., 149., 202.],
        ...,
        [123., 158., 194.],
        [134., 166., 203.],
        [135., 173., 220.]],

       [[ 76., 124., 180.],
        [110., 160., 198.],
        [114., 146., 209.],
        ...,
        [128., 161., 217.],
        [138., 153., 206.],
        [131., 150., 212.]],

       [[112., 133., 162.],
        [127., 148., 185.],
        [129., 156., 186.],
        ...,
        [130., 145., 187.],
        [119., 128., 167.],
        [110., 120., 168.]],

       ...,

       [[ 85., 104., 119.],
        [ 74.,  98., 132.],
        [ 87., 106., 155.],
        ...,
        [203., 197., 110.],
        [177., 178., 118.],
        [181., 197., 159.]],

       [[103., 128., 148.],
        [ 96., 117., 121.],
        [126., 140., 152.],
        ...,
        [175., 179., 118.],
        [157., 179., 169.],
        [120., 154., 178.]],

       [[119., 129.

In [305]:
len(list_img_convert)

100

In [337]:
list_label_convert = []
for i, label in enumerate(list_label):
    try:
        list_label_convert.append(tf.one_hot(int(label), 2, dtype=tf.float32))
    except TypeError:
        list_label_convert.append(list_label[0])
        

In [338]:
len(list_label_convert)

100

In [339]:
dataset_transformed = tf.data.Dataset.from_tensor_slices((list_img_convert, list_label_convert))
train_pipeline = dataset_transformed.shuffle(1024).repeat().batch(64).prefetch(2)

In [340]:
list_test = list(pcam['test'])[:50]

In [341]:
test_img = [list_test[i]['image'] for i in range(len(list_test))]
test_label = [list_test[i]['label'] for i in range(len(list_test))]

In [342]:
test_img_convert = []
for i, img in enumerate(test_img):
    test_img_convert.append(preprocess_content(img))
test_label_convert = []
for i, label in enumerate(test_label):
    try:
        test_label_convert.append(tf.one_hot(int(label), 2, dtype=tf.float32))
    except TypeError:
        test_label_convert.append(list_label[0])

<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD8A5931F0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCDE0062490>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD8A5937C0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD8A5937C0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD8A5937C0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCD8A5931F0>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF85553790>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF85553910>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF85553910>
shape (96, 96, 3)
shape2 (96, 96, 3)
<PIL.Image.Image image mode=RGB size=512x512 at 0x7FCF85553F40>
shape (96, 96, 3)
shape2 (9

In [345]:
test_transformed = tf.data.Dataset.from_tensor_slices((test_img_convert, test_label_convert))
validation_pipeline = dataset_transformed.shuffle(1024).repeat().batch(64).prefetch(2)

In [346]:
hist = model.fit(train_pipeline,
                 validation_data=validation_pipeline,
                 verbose=2, epochs=5, steps_per_epoch=4096, validation_steps=256)

Epoch 1/5
4096/4096 - 1589s - loss: nan - accuracy: 0.5700 - val_loss: nan - val_accuracy: 0.5702
Epoch 2/5
4096/4096 - 3366s - loss: nan - accuracy: 0.5700 - val_loss: nan - val_accuracy: 0.5700
Epoch 3/5
4096/4096 - 1724s - loss: nan - accuracy: 0.5700 - val_loss: nan - val_accuracy: 0.5701
Epoch 4/5
4096/4096 - 1671s - loss: nan - accuracy: 0.5700 - val_loss: nan - val_accuracy: 0.5703
Epoch 5/5
4096/4096 - 7056s - loss: nan - accuracy: 0.5700 - val_loss: nan - val_accuracy: 0.5699


In [271]:
import tensorflow_hub as hub
import glob
import functools
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
style_images = glob.glob("/Users/melody.zhao/Desktop/CS153-Computer-Vision/style_transfer_imgs/*")
style_image = load_img(style_images[2])

def dataset_maker(image, label):
#     image, label = dataset_obj[0],dataset_obj[1]
#     image, label = image["args_0:0"], label["args_1:0"]
#     image = preprocess_content(image)
    print(tf.convert_to_tensor(image))
    print(label)
#     print(tf.constant(image))
    image = hub_model(tf.constant(image), tf.constant(style_image))[0]
    print('here2')
    print(image)
    image = tensor_to_image(image)
#     image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

In [233]:
len(list_label)

100

In [234]:
len(list_img_convert)

100

In [235]:
tf.constant(list_img_convert[0])

<tf.Tensor: shape=(1, 512, 512, 3), dtype=float32, numpy=
array([[[[0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         ...,
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614]],

        [[0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         ...,
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614]],

        [[0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         [0.9686275 , 0.8078432 , 0.8705883 ],
         ...,
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614],
         [0.9803922 , 0.8196079 , 0.89019614]],

        ...,

        [[0.7607844 , 0.6666667 , 

In [236]:
dataset = tf.data.Dataset.from_tensor_slices((list(list_img_convert), list_label))
print(dataset)
# dataset = dataset.map(dataset_maker)

<TensorSliceDataset shapes: ((1, 512, 512, 3), ()), types: (tf.float32, tf.int64)>


In [241]:
for img, label in dataset:
    print(img)
    print('----')
    print(label)
    break

tf.Tensor(
[[[[0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   ...
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]]

  [[0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   ...
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]]

  [[0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   [0.9686275  0.8078432  0.8705883 ]
   ...
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]
   [0.9803922  0.8196079  0.89019614]]

  ...

  [[0.7607844  0.6666667  0.7294118 ]
   [0.7607844  0.6666667  0.7294118 ]
   [0.7607844  0.6666667  0.7294118 ]
   ...
   [0.8705883  0.8078432  0.85098046]
   [0.8705883  0.8078432  0.85098046]
   [0.8705883  0.8078432  0.85098046]]

  [[0.7607844  0.6666667  0.729411

In [238]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [272]:
dataset = dataset.map(dataset_maker)

Tensor("args_0:0", shape=(1, 512, 512, 3), dtype=float32)
Tensor("args_1:0", shape=(), dtype=int64)


TypeError: in user code:

    /var/folders/sc/4krr0g1j1fsg07slzqv1h0980000gr/T/ipykernel_77165/1070703169.py:14 dataset_maker  *
        image = hub_model(tf.constant(image), tf.constant(style_image))[0]
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py:271 constant  **
        
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py:288 _constant_impl
        
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/tensor_util.py:457 make_tensor_proto
        
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/tensor_util.py:334 _AssertCompatible
        

    TypeError: Expected any non-tensor type, got a tensor instead.


In [35]:
dataset.shuffle(1024).repeat().batch(64).prefetch(2)

<PrefetchDataset shapes: ((None, 1, 512, 512, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [86]:
output = map(convert_sample, list_train)


# print(list(pcam['train'])map(convert_sample))
# print(pcam['train'])
# print(tf.map_fn(convert_sample, pcam['train']))

In [91]:
print(pcam['validation'].map(convert_sample,
                                        num_parallel_calls=8))
output.shuffle(1024).repeat().batch(64).prefetch(2)

<ParallelMapDataset shapes: ((96, 96, 3), (2,)), types: (tf.float32, tf.float32)>


AttributeError: 'map' object has no attribute 'shuffle'

In [81]:
# shuffle the training data with a shuffle buffer 
# batches of 64 patches for training and 128 for validation
# prefetch 2 batches such that we can get batches during training on the GPU

train_pipeline = list(pcam['train']).map(convert_sample,
                                   num_parallel_calls=8).shuffle(1024).repeat().batch(64).prefetch(2)
# train_pipeline = tf.map_fn(convert_sample, pcam['train']).shuffle(1024).repeat().batch(64).prefetch(2)
valid_pipeline = pcam['validation'].map(convert_sample,
                                        num_parallel_calls=8).repeat().batch(128).prefetch(2)

OperatorNotAllowedInGraphError: in user code:

    /var/folders/sc/4krr0g1j1fsg07slzqv1h0980000gr/T/ipykernel_74745/2708019344.py:3 convert_sample  *
        image = preprocess_content(image)
    /var/folders/sc/4krr0g1j1fsg07slzqv1h0980000gr/T/ipykernel_74745/501126989.py:8 preprocess_content  *
        long_dim = max(shape)
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:520 __iter__  **
        self._disallow_iteration()
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:513 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /Users/melody.zhao/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:489 _disallow_when_autograph_enabled
        raise errors.OperatorNotAllowedInGraphError(

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying

In [7]:
hist = model.fit(train_pipeline,
                 validation_data=valid_pipeline,
                 verbose=2, epochs=5, steps_per_epoch=4096, validation_steps=256)

Epoch 1/5


2021-11-07 14:53:17.051142: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


4096/4096 - 1249s - loss: 0.4464 - accuracy: 0.7818 - val_loss: 0.3864 - val_accuracy: 0.8252
Epoch 2/5
4096/4096 - 1250s - loss: 0.3151 - accuracy: 0.8676 - val_loss: 0.3782 - val_accuracy: 0.8277
Epoch 3/5
4096/4096 - 1236s - loss: 0.2652 - accuracy: 0.8919 - val_loss: 0.3690 - val_accuracy: 0.8355
Epoch 4/5
4096/4096 - 1187s - loss: 0.2314 - accuracy: 0.9085 - val_loss: 0.3809 - val_accuracy: 0.8322
Epoch 5/5
4096/4096 - 1198s - loss: 0.2072 - accuracy: 0.9190 - val_loss: 0.3459 - val_accuracy: 0.8548


In [8]:
test_pipeline = pcam['test'].map(convert_sample, num_parallel_calls=8).batch(128).prefetch(2)
print("Test set accuracy is {0:.4f}".format(model.evaluate(test_pipeline, steps=128, verbose=0)[1]))


Test set accuracy is 0.8135


In [9]:
model.save("./patchcamelyon.hf5")


2021-11-07 16:52:55.329975: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./patchcamelyon.hf5/assets


INFO:tensorflow:Assets written to: ./patchcamelyon.hf5/assets
